In [5]:
from particle import Particle
from algorithms import BarnesHut, PairWise, FMM
from forces import Inverse
import numpy as np

## Barnes-Hut Error

In [ ]:
Ns = [10, 100, 1000]
ITERS = 40
THETAS = np.linspace(0, 1, 11)
G = 1

force = Inverse(G)
BH_algorithm = BarnesHut(force)
PW_algorithm = PairWise(force)

errors = {N: [] for N in Ns}
errors_std = {N: [] for N in Ns}

for N in Ns:
    print(N)
    for theta in THETAS:
        BH_algorithm.theta = theta

        temp_errors = []
        for _ in range(ITERS):
            particles = [Particle(charge=1/np.sqrt(N)) for _ in range(N)]

            BH_accelerations = BH_algorithm.calculate_accelerations(particles)
            PW_accelerations = PW_algorithm.calculate_accelerations(particles)

            diff = BH_accelerations - PW_accelerations
            error = np.mean(np.abs(diff))
            temp_errors.append(error)

        errors[N].append(np.mean(temp_errors))
        errors_std[N].append(np.std(temp_errors))       

## FMM Error

In [6]:
N = 100
MAX_DEPTH = 3
G = 1
PRECISIONS = np.arange(1, 31, 2)
ITERS = 10

force = Inverse(G)

PW_algorithm = PairWise(force)
particles = [[Particle(charge=1/np.sqrt(N)) for _ in range(N)] 
             for _ in range(ITERS)]

In [7]:
errors = []
errors_std = []

for precision in PRECISIONS:
    print(precision)
    temp_errors = []
    for i in range(ITERS):
        FMM_algorithm = FMM(MAX_DEPTH, precision, G)

        FMM_accelerations = FMM_algorithm.calculate_accelerations(particles[i])
        PW_accelerations = PW_algorithm.calculate_accelerations(particles[i])

        diff = FMM_accelerations - PW_accelerations
        error = np.mean(np.abs(diff))
        temp_errors.append(error)
    errors.append(np.mean(temp_errors))
    errors_std.append(np.std(temp_errors))

1
3
5
7
9
11
13
15
17
19
21
23
25
27
29
